In [ ]:
import numpy as np
from qiskit import QuantumCircuit, Aer, execute
import json

# 설정
n_qubits = 3
shots = 1024
max_iterations = 10
priority_threshold = 0.7
delta_angle = 0.25
n_trials = 5  # 실험 반복 횟수

sim = Aer.get_backend('qasm_simulator')
all_trials = []

# 회로 생성
def create_circuit(U):
    qc = QuantumCircuit(n_qubits, n_qubits)
    for i in range(n_qubits):
        qc.ry(U[i] * np.pi, i)
    qc.barrier()
    qc.measure(range(n_qubits), range(n_qubits))
    return qc

# 회로 실행
def run_circuit(qc):
    result = execute(qc, sim, shots=shots).result()
    return result.get_counts()

# 우선도 갱신
def update_priority(priority, counts):
    total = sum(counts.values())
    for bitstring, freq in counts.items():
        prob = freq / total
        for i, bit in enumerate(reversed(bitstring)):
            priority[i][int(bit)] += prob
    return priority

# 의도 계산
def compute_intent(priority):
    intent = []
    for qubit in priority:
        total = sum(qubit)
        dominant = max(qubit)
        intent.append(dominant / total if total > 0 else 0)
    return intent

# 큐빗 선택
def select_target(intent, modified_flags):
    intent_filtered = [
        val if (val >= priority_threshold and not modified_flags[i]) else 0.0
        for i, val in enumerate(intent)
    ]
    return int(np.argmax(intent_filtered)) if max(intent_filtered) >= priority_threshold else None

# 실험 단일 실행
def run_trial(trial_id):
    U = np.random.uniform(0.1, 0.9, n_qubits)
    priority = [[0.0, 0.0] for _ in range(n_qubits)]
    modified_flags = [False] * n_qubits
    trial_history = []

    for iteration in range(max_iterations):
        qc = create_circuit(U)
        counts = run_circuit(qc)
        priority = update_priority(priority, counts)
        intent = compute_intent(priority)
        target = select_target(intent, modified_flags)

        if target is not None:
            U[target] = min(U[target] + delta_angle, 1.0)
            modified_flags[target] = True

        trial_history.append({
            "trial": trial_id,
            "iteration": iteration,
            "params": U.tolist(),
            "counts": counts,
            "priority": priority,
            "intent": intent,
            "modified": target
        })

        if target is None and iteration > 0:
            break

    return trial_history

# 전체 실험 반복
for t in range(n_trials):
    print(f"▶ 실험 {t+1}/{n_trials} 실행 중...")
    trial_result = run_trial(t)
    all_trials.extend(trial_result)

# 결과 저장
with open("generalized_log.json", "w") as f:
    json.dump(all_trials, f, indent=2)
